In [1]:
import os
os.chdir('models/efficientdet/')
import src.model as mdl
import cv2
import numpy as np
import torch
import argparse
from torchvision import transforms
from src.dataset import CocoDataset, Resizer, Normalizer
from src.dataset import WaymoDataset
wd = WaymoDataset(cameras=['FRONT'],  transform=transforms.Compose([Normalizer(), Resizer()]))

In [ ]:
import train as train
parser = argparse.ArgumentParser(
    "EfficientDet: Scalable and Efficient Object Detection implementation by Signatrix GmbH")
parser.add_argument("--image_size", type=int, default=512, help="The common width and height for all images")
parser.add_argument("--batch_size", type=int, default=8, help="The number of images per batch")
parser.add_argument("--lr", type=float, default=1e-4)
parser.add_argument('--alpha', type=float, default=0.25)
parser.add_argument('--gamma', type=float, default=1.5)
parser.add_argument("--num_epochs", type=int, default=500)
parser.add_argument("--test_interval", type=int, default=1, help="Number of epoches between testing phases")
parser.add_argument("--es_min_delta", type=float, default=0.0,
                    help="Early stopping's parameter: minimum change loss to qualify as an improvement")
parser.add_argument("--es_patience", type=int, default=0,
                    help="Early stopping's parameter: number of epochs with no improvement after which training will be stopped. Set to 0 to disable this technique.")
parser.add_argument("--cam_view", type=str, default="FRONT", help="Camera view to train on [FRONT, SIDE_LEFT...]")
parser.add_argument("--log_path", type=str, default="tensorboard/efficientdet_waymo")
parser.add_argument("--saved_path", type=str, default="trained_models")
parser.add_argument("--train_depth", type=int, default=2)
parser.add_argument("--pretrained_model", type=str, default="", help="Path of pretrained model")
parser.add_argument("--backbone", type=str, default="efficientnet-b7")
parser.add_argument("-f", type=str, default="bam")

args = parser.parse_args()

# argparser = argparse.ArgumentParser()
# argparser.batch_size=1
train.train(args)
# train.get_args()


In [ ]:
import cv2
import numpy as np
import torch
ed = mdl.EfficientDet()
image = wd[0]['img'].numpy()
model(data['img'].cuda().permute(2, 0, 1).float().unsqueeze(dim=0))
# inputs = img.view([1, 3, 1280, 1920])
image_size=1280

# image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

height, width = image.shape[:2]
image = image.astype(np.float32) / 255
image[:, :, 0] = (image[:, :, 0] - 0.485) / 0.229
image[:, :, 1] = (image[:, :, 1] - 0.456) / 0.224
image[:, :, 2] = (image[:, :, 2] - 0.406) / 0.225
if height > width:
    scale = image_size / height
    resized_height = image_size
    resized_width = int(width * scale)
else:
    scale = image_size / width
    resized_height = int(height * scale)
    resized_width = image_size

image = cv2.resize(image, (resized_width, resized_height))

new_image = np.zeros((image_size, image_size, 3))
new_image[0:resized_height, 0:resized_width] = image
new_image = np.transpose(new_image, (2, 0, 1))
new_image = new_image[None, :, :, :]
new_image = torch.Tensor(new_image)

print(new_image.shape)
c3, c4, c5 = ed.backbone_net(new_image)
p3 = ed.conv3(c3)
p4 = ed.conv4(c4)
p5 = ed.conv5(c5)
p6 = ed.conv6(c5)
p7 = ed.conv7(p6)

features = [p3, p4, p5, p6, p7]
features = ed.bifpn(features)

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def transfer_learning(model, train_depth=2):
    mlist = next(m for m in model.children() if isinstance(m,torch.nn.modules.container.ModuleList))
    print(f"Num trainable params before:{count_parameters(mlist)}")
    layers_to_freeze = mlist[:-train_depth]
    for l in layers_to_freeze:
        freeze_params(l)
    print(f"Num trainable params after:{count_parameters(mlist)}")
    print(f"Layers 0 to {len(layers_to_freeze)-1} frozen, only training on last {train_depth} layers")

def freeze_params(l):
    for p in l.parameters():
        if p.requires_grad:
            p.requires_grad = False
# transfer_learning(ed.backbone_net.model)

In [ ]:
from data_utils import WaymoDataset
wd = WaymoDataset(heatmaps=False)

In [ ]:
wd[0]['labels']

In [3]:
import cv2
import numpy as np
import torch
ed = torch.load('trained_models/signatrix_efficientdet_coco.pth',
          map_location=torch.device('cpu')).module
# ed = mdl.EfficientDet()
# for i in range(15):
# image = wd[3]['img']
# x = ed(image.permute(2, 0, 1).float().unsqueeze(dim=0))
# x[0].shape
count_parameters(ed)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'src.model.EfficientDet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'src.loss.FocalLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'efficientnet_pytorch.model.EfficientNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.d

4499798

In [16]:
import waymo_open_dataset.metrics.python.detection_metrics as wod
wod.get_detection_metric_ops()

TypeError: get_detection_metric_ops() missing 10 required positional arguments: 'config', 'prediction_frame_id', 'prediction_bbox', 'prediction_type', 'prediction_score', 'prediction_overlap_nlz', 'ground_truth_frame_id', 'ground_truth_bbox', 'ground_truth_type', and 'ground_truth_difficulty'